In [1]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
import matplotlib.pyplot as plt
# import seaborn as sns
import pmlb as dsets
import numpy as np
import pickle as pkl
from os.path import join as oj
from copy import deepcopy
import pandas as pd
from numpy import array as arr
import time

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy import interpolate
from sklearn.tree import export_graphviz, DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.tree import plot_tree

import torch
from torch import nn
from util import t, to_sparse

import predict
from rf_to_dnn import Net

In [30]:
np.random.seed(13)
num_features = 4
N = 1000
X = np.random.rand(N, num_features)
# y = np.logical_xor(X[:, 0] > 0.5, X[:, 1] > 0.5)
y = np.random.rand(N)
# plt.scatter(X[:, 0], X[:, 1], c=y)
# plt.show()

rf = DecisionTreeRegressor(max_depth=50)
rf.fit(X, y)
print(rf.score(X, y), 'max depth', rf.get_depth())

# plt.scatter(X[:, 0], X[:, 1], c=rf.predict(X))
# plt.show()
# plot_tree(rf.estimators_[0])
# plot_tree(rf)
# plt.show()

0.9999999989409366


35

In [17]:
net = Net(rf)
b0, idxs0, w1, b1, idxs2 = net.extract_util_np()
idxs0_t, b0_t, lay1, b1_t, idxs2_t = \
    torch.Tensor(idxs0), torch.Tensor(b0), net.layers[1], \
    torch.Tensor(b1), torch.Tensor(idxs2)
# w1 = sparse.csr_matrix(w1)

In [18]:
X_t = torch.Tensor(X)
X_tg = deepcopy(torch.Tensor(X)).to('cuda')
netg = deepcopy(net).to('cuda')
net_s = deepcopy(net)
net_s.layers[1].weight.data = to_sparse(net.layers[1].weight)

In [28]:
with torch.no_grad():
    p1 = predict.f_torch_basic(net, X_t).detach().numpy()
    p2 = predict.f_np_basic(X, idxs0, b0, w1, b1, idxs2)
    p3 = predict.f_torch_indexing(X_t, idxs0, b0_t, lay1, b1_t, idxs2_t)
    # preds = forward_np(X[:10]) #, thresh, idxs, w, b)

t1: 1.77e-03, t2: 3.15e-02 t3: 8.90e-02
t1: 1.30e-02, t2: 4.39e-02 t3: 7.53e-02


t1: 1.43e-02, t2: 3.54e-02 t3: 9.17e-02


In [26]:
if np.isclose(rf.predict(X).flatten(), p3.flatten()).all():
    print('success!')
else:
    print('fail')

success!


In [ ]:
def f1(): rf.predict(X)
def f2(): 
    with torch.no_grad(): 
        net(X_t)
def f3(): 
    with torch.no_grad(): 
        forward_np(X)
        
print(f'rf: \t\t{t(f1):0.2e}\n\
dnn cpu: \t{t(f2):0.2e}\n\
dnn np: \t{t(f3):0.2e}\n')